## PREPROCESSING

### Install and Importing Library/Package

In [11]:
# Installing Library / Packages
%pip install tensorflow==2.17.1


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:

%pip install lime


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip install gensim


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)

Num GPUs Available:  0
2.17.1


In [28]:
import pandas as pd
import gensim
import numpy as np

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report

#Lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from lime.lime_text import IndexedString,IndexedCharacters
from lime.lime_base import LimeBase


### Membuat Model

#### Mengimport file hasil preprocessing

In [17]:
# Load CSV files into DataFrames
data_train = pd.read_csv('../datasets/output_preprocess/data_train.csv')
data_test = pd.read_csv('../datasets/output_preprocess/data_test.csv')
data_val = pd.read_csv('../datasets/output_preprocess/data_val.csv')

In [18]:
#Splitting the text from the labels
X_train = data_train['text']
y_train = data_train['label']

X_test = data_test['text']
y_test = data_test['label']

X_val = data_val['text']
y_val = data_val['label']

#### Encode Labels

In [19]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_val = le.transform(y_val)

In [20]:
#Convert the class vector (integers) to binary class matrix
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [21]:
MAX_LENGTH = 500

# Create a tokenizer
tokenizer = Tokenizer(oov_token='UNK')
# Fit the tokenizer on the training data
tokenizer.fit_on_texts(X_train)
# Convert text to sequences of tokens
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_val_seq = tokenizer.texts_to_sequences(X_val)
# Pad sequences to ensure equal length
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, truncating='pre')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, truncating='pre')
X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LENGTH, truncating='pre')

In [29]:
# Train Word2Vec model
sentences = [text.split() for text in X_train]
w2v_model = gensim.models.Word2Vec(sentences, vector_size=200, window=5, min_count=5, workers=4)

# Create embedding matrix
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 200))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

#### Membangun arsitekteur NN

In [30]:
# mendefinisikan vocabSize
vocabSize = len(tokenizer.word_index) + 1  # +1 for the padding token or OOV token

model = Sequential()
model.add(Embedding(vocabSize, 200, weights=[embedding_matrix])) # Use X_train_pad to get shape
model.add(Bidirectional(LSTM(256, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(6, activation='softmax'))


adam = Adam(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     2,697,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,697,200 (10.29 MB)

 Trainable params: 2,697,200 (10.29 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
#to stop the training when the loss starts to increase
callback = EarlyStopping(
    monitor="val_loss",      # Memantau loss pada data validasi
    patience=4,              # Menunggu 4 epoch sebelum menghentikan pelatihan jika val_loss tidak membaik
    restore_best_weights=True  # Mengembalikan bobot terbaik saat pelatihan dihentikan
)

In [ ]:
# Fit model
history = model.fit(X_train_pad,
                    y_train,
                    validation_data=(X_val_pad, y_val),
                    verbose=1,
                    batch_size=256,
                    epochs=10,
                    callbacks=[callback]
                   )

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 632s 10s/step - accuracy: 0.3117 - loss: 1.6313 - val_accuracy: 0.5876 - val_loss: 1.1642
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 665s 10s/step - accuracy: 0.7459 - loss: 0.7611 - val_accuracy: 0.9094 - val_loss: 0.2413
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 623s 10s/step - accuracy: 0.9499 - loss: 0.1329 - val_accuracy: 0.9254 - val_loss: 0.1600
Epoch 4/10
19/63 ━━━━━━━━━━━━━━━━━━━━ 6:57 9s/step - accuracy: 0.9756 - loss: 0.0588

In [ ]:
plot_model(model, show_shapes=True, dpi=80)

In [ ]:
#print the overall loss and accuracy
model.evaluate(X_test_pad, y_test)

In [ ]:
data_test['label'].value_counts()

In [ ]:
predicted = model.predict(X_test_pad)
y_pred = predicted.argmax(axis=-1)

print(classification_report(le.transform(data_test['label']), y_pred))

In [ ]:
# Visualize Loss & Accuracy

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('../model/emotion-classification.h5')

### Predicting diary input